# 在 Azure Kubernetes Service (AKS) 上使用 LLaMA-Factory Docker 部署 Llama3.1-8B 微调模型

本教程将指导您使用 LLaMA-Factory Docker 镜像在 AKS 上部署微调后的模型，实现一个可扩展的模型推理服务。

## 概述

LLaMA-Factory 是一个开源的大语言模型微调框架，提供了简单易用的工具来对 LLaMA、ChatGLM、Baichuan 等主流大模型进行高效的参数微调和训练

Llama3.1-8B 是 Meta 发布的一个高效的大型语言模型

Docker 是一个容器化平台，将应用程序及其依赖环境打包成轻量级、可移植的容器，实现"一次构建，处处运行"的部署方式。

Azure Kubernetes Service (AKS) 是微软 Azure 提供的托管 Kubernetes 服务，简化了容器化应用的部署、管理和扩展，无需用户管理 Kubernetes 控制平面。



## 前置条件

### 硬件要求
- **CPU**: 至少 8 核心（推荐 16 核心或以上）
- **内存**: 最少 16GB RAM（推荐 32GB 或以上）
- **存储**: 至少 50GB 可用空间
- **GPU**: NVIDIA GPU（至少 16GB 显存）

### 软件要求
- **操作系统**: Ubuntu 20.04/22.04 LTS 或 CentOS 7/8
- **Python**: 3.8 或更高版本
- **Docker**: 20.10 或更高版本
- **CUDA**: 11.8 或更高版本（如果使用 GPU）

### 系统要求
- Azure 订阅（如果没有，请创建[免费账户](https://azure.microsoft.com/free/)）
- Azure CLI 2.50.0 或更高版本
- kubectl 命令行工具
- Docker 命令行工具
- 至少 16GB RAM 的开发机器

### 安装必要工具

首先，我们需要安装和配置必要的工具。以下命令将安装 Azure CLI、kubectl 和相关扩展：

In [ ]:
# 安装 Azure CLI (如果尚未安装)
!curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash

# 验证安装
!az --version

# 安装 kubectl
!az aks install-cli

# 安装 k8s-extension 扩展
!az extension add --name k8s-extension

# 更新扩展到最新版本
!az extension update --name k8s-extension

### 登录 Azure

使用以下命令登录到 Azure 并设置默认订阅：

In [ ]:
import os

# 设置环境变量
os.environ["RESOURCE_GROUP"] = "lenovo_vm_rg"
os.environ["LOCATION"] = "eastus2"
os.environ["AKS_CLUSTER_NAME"] = "llama-aks-cluster"
os.environ["ACR_NAME"] = f"llamaacr"
os.environ["NAMESPACE"] = "llama-inference"

# 显示配置
print(f"Resource Group: {os.environ['RESOURCE_GROUP']}")
print(f"Location: {os.environ['LOCATION']}")
print(f"AKS Cluster: {os.environ['AKS_CLUSTER_NAME']}")
print(f"ACR Name: {os.environ['ACR_NAME']}")
print(f"Namespace: {os.environ['NAMESPACE']}")

## 创建 AKS 集群

### 1. 创建资源组

Azure 中的资源组是一个逻辑容器, 它允许您根据资源的生命周期和安全需求将这些资源作为一个整体进行管理。资源组中的资源可以包括虚拟机、存储账户和虚拟网络等。

首先，我们需要创建一个资源组来管理所有相关资源：

In [ ]:
# 创建资源组
!az group create \
    --name $RESOURCE_GROUP \
    --location $LOCATION

# 验证资源组创建
!az group show --name $RESOURCE_GROUP

### 2. 创建 AKS 集群

创建一个适合运行 LLaMA 模型的 AKS 集群：

In [ ]:
!az aks create \
    --resource-group $RESOURCE_GROUP \
    --name $AKS_CLUSTER_NAME \
    --node-count 2 \
    --node-vm-size Standard_NC6s_v3 \
    --enable-managed-identity \
    --generate-ssh-keys \
    --enable-addons monitoring \
    --enable-cluster-autoscaler \
    --min-count 1 \
    --max-count 3 \
    --node-osdisk-size 100

参数说明：

- **`--resource-group $RESOURCE_GROUP`** 
  - 指定资源组

- **`--name $AKS_CLUSTER_NAME`** 
  - 指定 AKS 集群的名称

- **`--node-count 2`**
  - 设置初始节点数量为 2 个
  - 这是集群启动时的默认节点池大小

- **`--node-vm-size Standard_NC6s_v3`**
  - 指定节点虚拟机的规格为 Standard_NC6s_v3
  - 包含 6 个 vCPU、56 GB RAM 和 1 个 GPU

- **`--node-osdisk-size 100`**
  - 设置每个节点的操作系统磁盘大小为 100 GB
  - 默认通常是 30 GB，100 GB 提供更多存储空间

- **`--enable-managed-identity`**
  - 启用托管身份
  - Azure 会自动创建和管理服务主体，简化身份验证配置

- **`--generate-ssh-keys`**
  - 自动生成 SSH 密钥对
  - 用于访问集群节点（如果需要的话）

- **`--enable-addons monitoring`**
  - 启用 Azure Monitor 容器监控
  - 提供集群和容器的监控、日志收集功能

- **`--enable-cluster-autoscaler`**
  - 启用集群自动缩放器
  - 根据工作负载需求自动调整节点数量

- **`--min-count 1`**
  - 设置自动缩放的最小节点数为 1

- **`--max-count 3`**
  - 设置自动缩放的最大节点数为 3


验证集群连接


In [ ]:
!kubectl get nodes

返回结果包含：
- NAME - 节点名称
- STATUS - 节点状态（Ready/NotReady）
- ROLES - 节点角色（master/worker）
- AGE - 节点运行时间
- VERSION - Kubernetes 版本

返回示例：
```shell
NAME                                STATUS   ROLES   AGE   VERSION
aks-nodepool1-12345678-vmss000000   Ready    agent   1h    v1.27.7
aks-nodepool1-12345678-vmss000001   Ready    agent   1h    v1.27.7
```

获取 AKS 凭据

In [ ]:
%%bash
az aks get-credentials \
    --resource-group $RESOURCE_GROUP \
    --name $AKS_CLUSTER_NAME \
    --overwrite-existing


参数说明：
- **`az aks get-credentials`**
  - Azure CLI 命令，用于下载并配置 AKS 集群的 kubeconfig 文件
  - 配置 kubectl 以连接到指定的 AKS 集群

- **`--resource-group $RESOURCE_GROUP`**
  - 指定 AKS 集群所在的 Azure 资源组
  - `$RESOURCE_GROUP` 是环境变量，需要与创建集群时使用的资源组一致

- **`--name $AKS_CLUSTER_NAME`**
  - 指定要获取凭据的 AKS 集群名称
  - `$AKS_CLUSTER_NAME` 是环境变量，需要与实际的集群名称一致

- **`--overwrite-existing`**
  - 如果本地已存在同名的集群配置，则覆盖现有配置
  - 不使用此参数时，如果存在同名配置会报错
  - 确保获取到最新的集群凭据

### 3. 安装 GPU 驱动程序（如果使用 GPU 节点）

为 AKS 集群配置 GPU 支持：

In [ ]:
# 安装 NVIDIA GPU 设备插件
!kubectl apply -f https://raw.githubusercontent.com/NVIDIA/k8s-device-plugin/v0.14.1/nvidia-device-plugin.yml

# 验证 GPU 节点
!kubectl get nodes -o wide

# 检查 GPU 资源
!kubectl describe nodes | grep -A 5 "Capacity\|Allocatable" | grep nvidia

## 配置 Container Registry

Azure Container Registry (ACR) 是 Azure 提供的私有 Docker 镜像仓库服务，用于存储、管理和部署容器镜像，让 AKS 等服务可以安全地拉取自定义应用镜像。

### 1. 创建 Azure Container Registry

In [ ]:
# 创建 Azure Container Registry
!az acr create \
    --name $ACR_NAME \
    --resource-group $RESOURCE_GROUP \
    --location $LOCATION \
    --sku Standard \
    --admin-enabled true

参数说明：

- **`az acr create`** - Azure CLI 命令，用于创建新的 Azure 容器注册表

- **`--name $ACR_NAME`** - 指定容器注册表的名称，使用环境变量 `$ACR_NAME`，名称必须全局唯一且只能包含字母和数字

- **`--resource-group $RESOURCE_GROUP`** - 指定要在其中创建 ACR 的 Azure 资源组，使用环境变量 `$RESOURCE_GROUP`

- **`--location $LOCATION`** - 指定 ACR 的部署位置/区域，使用环境变量 `$LOCATION`，如 `eastus`、`westus2` 等

- **`--sku Standard`** - 设置 ACR 的定价层为 Standard，提供中等性能和功能，包括 webhook、geo-replication 等功能

- **`--admin-enabled true`** - 启用管理员用户账户，允许使用用户名/密码进行身份验证，便于与 Docker CLI 和其他工具集成

登录到ACR

In [ ]:
!az acr login --name $ACR_NAME

获取 ACR 登录服务器

In [ ]:
%%bash
ACR_LOGIN_SERVER=$(az acr show --name $ACR_NAME --resource-group $RESOURCE_GROUP --query loginServer -o tsv)
echo "ACR Login Server: $ACR_LOGIN_SERVER"

将 ACR 与 AKS 集群关联

In [ ]:

!az aks update \
    --name $AKS_CLUSTER_NAME \
    --resource-group $RESOURCE_GROUP \
    --attach-acr $ACR_NAME

### 2. 准备 LLaMA-Factory Docker 镜像

我们可以直接使用官方的 LLaMA-Factory 镜像，或者自定义镜像：

#### 2.1 克隆 LLaMa-Factory 仓库

In [ ]:
# 克隆 LLaMA-Factory 项目
!git clone https://github.com/hiyouga/LLaMA-Factory.git
!cd LLaMA-Factory

# 查看项目结构
!ls -la LLaMA-Factory/

### 2.2 构建镜像

In [ ]:
%%bash
cd LLaMA-Factory/docker/docker-cuda/

# 构建镜像
docker build -f ./docker/docker-cuda/Dockerfile \
    --build-arg PIP_INDEX=https://pypi.org/simple \
    --build-arg EXTRAS=metrics \
    -t llamafactory:latest .

参数说明:
- **`docker build`** - Docker 命令，用于构建容器镜像

- **`-f ./docker/docker-cuda/Dockerfile`** - 指定构建使用的 Dockerfile 文件路径

- **`--build-arg PIP_INDEX=https://pypi.org/simple`** - 设置构建参数，指定 pip 包索引源为官方 PyPI 源

- **`--build-arg EXTRAS=metrics`** - 设置构建参数，指定安装额外的 metrics 功能包

- **`-t llamafactory:latest`** - 为构建的镜像指定标签名称和版本，这里命名为 llamafactory，版本为 latest

- **`.`** - 指定构建上下文为当前目录，Docker 会使用当前目录下的文件作为构建资源


测试容器服务

In [ ]:
!docker compose exec llamafactory bash

### 2.3 推送到 ACR

In [ ]:
%%bash
# 获取 ACR 登录服务器地址
ACR_LOGIN_SERVER=$(az acr show --name $ACR_NAME --resource-group $RESOURCE_GROUP --query loginServer -o tsv)

In [ ]:


# 推送到 ACR
!docker push ${ACR_LOGIN_SERVER}/llamafactory:latest

# 验证镜像推送
!az acr repository list --name $ACR_NAME

## 创建 LLaMA-Factory 部署文件

### 1. 创建 Namespace 和 ConfigMap

创建配置文件目录

In [1]:
!mkdir -p k8s-manifests

In [ ]:
# 创建命名空间
!kubectl create namespace $NAMESPACE

# 验证命名空间创建
!kubectl get namespaces

#### 1.1 创建 `configmap.yaml`

`configmap.yaml` 是用于存储应用配置数据的配置文件，将配置信息（如环境变量、配置文件内容）从应用代码中分离出来，以键值对形式存储，供 Pod 以环境变量或挂载文件的方式使用。

In [2]:
%%writefile k8s-manifests/configmap.yaml
apiVersion: v1
kind: ConfigMap
metadata:
  name: llama-config
  namespace: llama-inference
data:
  # API 服务器监听地址，0.0.0.0 表示监听所有网络接口
  API_HOST: "0.0.0.0"
  # API 服务器监听端口，用于接收推理请求
  API_PORT: "8000"
  # Web UI 服务监听地址，0.0.0.0 表示监听所有网络接口
  WEBUI_HOST: "0.0.0.0" 
  # Web UI 服务监听端口，用于访问 Web 界面
  WEBUI_PORT: "7860"
  # 要加载的模型名称，需要替换为实际使用的模型
  MODEL_NAME: "your-model-name"  # 替换为您的模型名称
  # 微调适配器路径，如果有 LoRA 等微调适配器则指定路径
  ADAPTER_NAME: "your-adapter-path"  # 如果有微调适配器的话
  # 量化位数，4 表示使用 4-bit 量化以节省内存
  QUANTIZATION_BIT: "4"
  # 对话模板类型，用于格式化输入输出
  TEMPLATE: "default"
  # Hugging Face 镜像地址，用于下载模型和数据
  HF_ENDPOINT: "https://hf-mirror.com"

Writing k8s-manifests/configmap.yaml


#### 1.2 创建 `deployment.yaml`

`deployment.yaml` 是用于定义和管理应用部署的配置文件，描述了应用容器的运行规格、副本数量、更新策略等，Kubernetes 会根据这个文件自动创建、维护和扩展 Pod，确保应用始终按预期状态运行。

In [3]:
%%writefile k8s-manifests/deployment.yaml
# Kubernetes Deployment 资源定义
apiVersion: apps/v1
kind: Deployment
metadata:
  name: llama-deployment
  namespace: llama-inference  # 部署到指定命名空间
  labels:
    app: llama  # 标签用于资源标识和选择器匹配
spec:
  replicas: 1  # 副本数量，只运行一个实例
  selector:
    matchLabels:
      app: llama  # 选择器，匹配具有 app=llama 标签的 Pod
  template:
    metadata:
      labels:
        app: llama  # Pod 模板标签
    spec:
      containers:
      - name: llamafactory
        # 容器镜像，从 ACR 拉取 llamafactory 镜像
        image: ${ACR_LOGIN_SERVER}/llamafactory:latest
        ports:
        - containerPort: 8000
          name: api      # API 服务端口
        - containerPort: 7860
          name: webui    # Web UI 端口
        env:
        # 指定使用第一个 GPU 设备
        - name: CUDA_VISIBLE_DEVICES
          value: "0"
        # Hugging Face 模型缓存目录
        - name: HF_HOME
          value: "/app/models"
        # Hugging Face Hub 缓存目录
        - name: HF_HUB_CACHE
          value: "/app/models"
        # 从 ConfigMap 导入环境变量
        envFrom:
        - configMapRef:
            name: llama-config
        # 容器启动命令
        command: ["/bin/bash"]
        args:
        - -c
        - |
          # 启动 API 服务，后台运行
          llamafactory-cli api \
            --model_name_or_path $(MODEL_NAME) \
            --template $(TEMPLATE) \
            --quantization_bit $(QUANTIZATION_BIT) \
            --host $(API_HOST) \
            --port $(API_PORT) &
          
          # 启动 Web UI，后台运行
          llamafactory-cli webui \
            --host $(WEBUI_HOST) \
            --port $(WEBUI_PORT) &
          
          # 等待所有后台进程，保持容器运行
          wait
        resources:
          requests:  # 资源请求，Pod 调度时的最小要求
            memory: "8Gi"          # 请求 8GB 内存
            cpu: "2"               # 请求 2 核 CPU
            nvidia.com/gpu: 1      # 请求 1 个 GPU
          limits:    # 资源限制，Pod 运行时的最大使用量
            memory: "16Gi"         # 限制最大 16GB 内存
            cpu: "4"               # 限制最大 4 核 CPU
            nvidia.com/gpu: 1      # 限制最大 1 个 GPU
        volumeMounts:
        # 挂载模型缓存目录
        - name: model-cache
          mountPath: /app/models
        # 挂载共享内存，用于 GPU 和进程间通信
        - name: shm
          mountPath: /dev/shm
        # 就绪探针，检查容器是否准备好接收流量
        readinessProbe:
          httpGet:
            path: /docs          # 检查 API 文档端点
            port: 8000
          initialDelaySeconds: 300  # 初始延迟 5 分钟
          periodSeconds: 30         # 每 30 秒检查一次
          timeoutSeconds: 10        # 超时时间 10 秒
        # 存活探针，检查容器是否仍在运行
        livenessProbe:
          httpGet:
            path: /docs          # 检查 API 文档端点
            port: 8000
          initialDelaySeconds: 600  # 初始延迟 10 分钟
          periodSeconds: 60         # 每 60 秒检查一次
          timeoutSeconds: 30        # 超时时间 30 秒
      volumes:
      # 模型缓存卷，用于存储下载的模型文件
      - name: model-cache
        emptyDir:
          sizeLimit: 50Gi  # 限制大小为 50GB
      # 共享内存卷，用于高性能计算
      - name: shm
        emptyDir:
          medium: Memory   # 使用内存作为存储介质
          sizeLimit: 8Gi   # 限制大小为 8GB
      # 容忍度，允许调度到有 GPU 污点的节点
      tolerations:
      - key: "sku"
        operator: "Equal"
        value: "gpu"
        effect: "NoSchedule"
      # 节点选择器，只调度到有 nvidia GPU 的节点
      nodeSelector:
        accelerator: nvidia

Writing k8s-manifests/deployment.yaml


#### 1.3 创建 `service.yaml`

`service.yaml` 是用于定义服务网络访问的配置文件，为一组 Pod 提供稳定的网络端点和负载均衡，使得应用可以通过固定的 IP 地址和端口被访问，而无需关心底层 Pod 的动态变化。

In [4]:
%%writefile k8s-manifests/service.yaml
# 第一个服务：LoadBalancer 类型，对外暴露 API 和 WebUI
apiVersion: v1
kind: Service
metadata:
  name: llama-service
  namespace: llama-inference  # 部署到指定命名空间
  labels:
    app: llama  # 服务标签
spec:
  type: LoadBalancer  # 负载均衡器类型，会创建外部 IP 供外部访问
  ports:
  - name: api
    port: 8000          # 外部访问端口
    targetPort: 8000    # Pod 内部端口
    protocol: TCP       # 协议类型
  - name: webui
    port: 7860          # Web UI 外部访问端口
    targetPort: 7860    # Pod 内部 Web UI 端口
    protocol: TCP       # 协议类型
  selector:
    app: llama  # 选择器，匹配 app=llama 标签的 Pod

---
# 第二个服务：ClusterIP 类型，仅集群内部访问 API 服务
apiVersion: v1
kind: Service
metadata:
  name: llama-api-service
  namespace: llama-inference  # 部署到指定命名空间
  labels:
    app: llama  # 服务标签
spec:
  type: ClusterIP  # 集群内部 IP，只能从集群内部访问
  ports:
  - name: api
    port: 8000          # 集群内部访问端口
    targetPort: 8000    # Pod 内部端口
    protocol: TCP       # 协议类型
  selector:
    app: llama  # 选择器，匹配 app=llama 标签的 Pod

---
# 第三个服务：ClusterIP 类型，仅集群内部访问 Web UI 服务
apiVersion: v1
kind: Service
metadata:
  name: llama-webui-service
  namespace: llama-inference  # 部署到指定命名空间
  labels:
    app: llama  # 服务标签
spec:
  type: ClusterIP  # 集群内部 IP，只能从集群内部访问
  ports:
  - name: webui
    port: 7860          # 集群内部访问端口
    targetPort: 7860    # Pod 内部 Web UI 端口
    protocol: TCP       # 协议类型
  selector:
    app: llama  # 选择器，匹配 app=llama 标签的 Pod

Writing k8s-manifests/service.yaml


#### 1.4 更新 `deployment.yaml` 镜像地址

In [ ]:
%%bash
ACR_LOGIN_SERVER=$(az acr show --name $ACR_NAME --resource-group $RESOURCE_GROUP --query loginServer -o tsv)

In [ ]:
# 替换部署文件中的镜像地址
!sed -i "s/\${ACR_LOGIN_SERVER}/$ACR_LOGIN_SERVER/g" k8s-manifests/deployment.yaml

# 应用所有配置文件
!kubectl apply -f k8s-manifests/

# 检查部署状态
!kubectl get pods -n $NAMESPACE
!kubectl get services -n $NAMESPACE

#### 1.5 等待服务就绪

In [ ]:
# 等待 Pod 就绪
!kubectl wait --for=condition=ready pod -l app=llama -n $NAMESPACE --timeout=600s

# 查看详细状态
!kubectl describe pods -l app=llama -n $NAMESPACE

# 查看日志
!kubectl logs -l app=llama -n $NAMESPACE --tail=50

# 检查服务状态
!kubectl get svc -n $NAMESPACE -o wide

获取外部IP

In [ ]:
# 等待外部 IP 分配
!kubectl get service llama-service -n $NAMESPACE --watch

# 获取外部 IP（这可能需要几分钟时间）
!kubectl get service llama-service -n $NAMESPACE -o jsonpath={.status.loadBalancer.ingress[0].ip}

external_ip 是终端输出的外部IP

In [ ]:
# 测试 API 健康状态
!curl -X GET http://{external_ip}:8000/v1/models

# 测试文档页面
!curl -X GET http://{external_ip}:8000/docs

### 2. 使用 Python 测试 API

In [ ]:
import requests
import json

# API 端点
api_url = f"http://{external_ip}:8000/v1/chat/completions"

# 测试数据
test_cases = [
    {
        "model": "microsoft/Phi-4",
        "messages": [
            {"role": "user", "content": "解释什么是量子计算？"}
        ],
        "max_tokens": 200,
        "temperature": 0.7
    },
    {
        "model": "microsoft/Phi-4", 
        "messages": [
            {"role": "user", "content": "写一个 Python 函数计算斐波那契数列"}
        ],
        "max_tokens": 300,
        "temperature": 0.5
    }
]

# 发送测试请求
for i, test_data in enumerate(test_cases, 1):
    print(f"\n=== 测试 {i} ===")
    print(f"问题: {test_data['messages'][0]['content']}")
    print("-" * 50)
    
    try:
        response = requests.post(
            api_url,
            headers={"Content-Type": "application/json"},
            json=test_data,
            timeout=120
        )
        
        if response.status_code == 200:
            result = response.json()
            content = result.get('choices', [{}])[0].get('message', {}).get('content', '')
            usage = result.get('usage', {})
            
            print(f"回答: {content}")
            print(f"令牌使用: {usage}")
        else:
            print(f"请求失败: {response.status_code}")
            print(f"错误信息: {response.text}")
            
    except requests.exceptions.RequestException as e:
        print(f"请求异常: {e}")
    except Exception as e:
        print(f"其他错误: {e}")

### 3. 查看日志与监控

In [ ]:
# 查看 Pod 资源使用情况
!kubectl top pods -n $NAMESPACE

# 查看节点资源使用情况  
!kubectl top nodes

# 查看 Pod 事件
!kubectl get events -n $NAMESPACE --sort-by='.lastTimestamp'

# 实时监控日志
!kubectl logs -f -l app=llama -n $NAMESPACE

## 清理资源

当您完成测试后，记得清理创建的资源以避免不必要的费用：

In [ ]:
# 删除部署
!kubectl delete -f k8s-manifests/

# 删除命名空间
!kubectl delete namespace $NAMESPACE

# 删除 ACR
!az acr delete \
    --name $ACR_NAME \
    --resource-group $RESOURCE_GROUP \
    --yes

# 删除 AKS 集群
!az aks delete \
    --name $AKS_CLUSTER_NAME \
    --resource-group $RESOURCE_GROUP \
    --yes

# 删除资源组（这将删除所有剩余资源）
!az group delete \
    --name $RESOURCE_GROUP \
    --yes